
# Build a basic ME model

We will try to build an ME model from the NC_000913.2 Genbank file, the iJO1366 M model, and the complex reconstruction from iJL1650-ME

In [1]:
# python imports
import re
import json
from os.path import join

# third party imports
import pandas
import escher
import cobra.test

# ecoli me
from ecolime.flat_files import *

from minime import *
from minime.util.building import build_reactions_from_genbank, add_transcription_reaction
from minime.solve.algorithms import binary_search, fva, solve_at_growth_rate
from minime.solve.symbolic import compile_expressions

## Begin by loading metabolites

In [2]:
def fix_id(id_str):
    return id_str.replace("_DASH_", "__")

In [3]:
me = MEmodel("iJO1366-ME")
me.id = "iJO1366-ME"
me.compartments = {"p": "Periplasm", "e": "Extra-organism",
                   "c": "Cytosol"}
compartment_lookup = {v: k for k, v in me.compartments.items()}

In [4]:
met_info = pandas.read_csv(
    join(ecoli_files_dir, "metabolites.txt"),
    delimiter="\t", header=None, index_col=0,
    names=["id", "name", "formula", "compartment", "data_source"])

In [5]:
for met_id in met_info.index:
    fixed_id = fix_id(met_id)
    for compartment in met_info.compartment[met_id].split("AND"):
        compartment = compartment.strip()
        if compartment == "No_Compartment":
            print "Assigned %s to e" % met_id
            compartment = me.compartments["e"]
        metab = Metabolite(fixed_id + "_" + compartment_lookup[compartment])
        metab.name = met_info.name[met_id]
    
        metab.formula = met_info.formula[met_id]
        me.add_metabolites(metab)

Assigned tl to e
Assigned hemed to e
Assigned pqq to e
Assigned dpm to e
Assigned 23bpg to e
Assigned tqn to e


Add generic ions to simplify complexes

In [6]:
divalent_list = divalent_list
monovalent_list = monovalent_list
comp = "_c"  # compartment
# Divalent
div = cobra.Metabolite('generic_divalent' + comp)
div.name = 'Generic divalent ion'
me.add_metabolites(div)
for ion in divalent_list:
    rxn1 = cobra.Reaction(ion + comp + '_to_generic')
    ion_dict = {}
    met1 = me.metabolites.get_by_id(ion + comp)
    ion_dict[met1] = -1
    ion_dict[div] = 1
    rxn1.add_metabolites(ion_dict)
    me.add_reaction(rxn1)
# Monovalent
mono = cobra.Metabolite('generic_monovalent' + comp)
mono.name = 'Generic monovalent ion'
for ion in monovalent_list:
    rxn2 = cobra.Reaction(ion + comp + '_to_generic')
    ion_dict = {}
    met2 = me.metabolites.get_by_id(ion + comp)
    ion_dict[met2] = -1
    ion_dict[mono] = 1
    rxn2.add_metabolites(ion_dict)
    me.add_reaction(rxn2)

## Build Metabolic reactions

In [7]:
reaction_info = get_reaction_info_frame()
reaction_dict = get_reaction_matrix_dict()

In [8]:
for r_id in reaction_info.index:
    reaction = StoichiometricData(r_id, me)
    reaction._stoichiometry = {fix_id(k): v\
                               for k, v in reaction_dict[r_id].items()}
    reaction.lower_bound = \
        -1000. if reaction_info.is_reversible[r_id] else 0.
    reaction.upper_bound = 1000.

Also make a dummy reaction

In [9]:
dummy = StoichiometricData("dummy_reaction", me)
dummy.lower_bound = 0
dummy.upper_bound = 1000
dummy._stoichiometry = {}

Boundary Reactions

In [10]:
sources_sinks = pandas.read_csv(join(ecoli_files_dir, "reaction_matrix_sources_and_sinks.txt"), delimiter="\t",
                                header=None, names=["rxn_id", "met_id", "compartment", "stoic"], index_col=1)
sources_sinks.index = [fix_id(i) for i in sources_sinks.index]

source_amounts = pandas.read_csv(join(ecoli_files_dir, "exchange_bounds.txt"),
                                 delimiter="\t", index_col=0, names=["met_id", "amount"])
source_amounts.index = [fix_id(i) for i in source_amounts.index]


for met_id in sources_sinks.index:
    model_met_id = met_id + "_" + compartment_lookup[sources_sinks.compartment[met_id]]
    # EX_ or DM_ + met_id
    reaction_id = sources_sinks.rxn_id[met_id][:3] + model_met_id
    reaction = cobra.Reaction(reaction_id)
    me.add_reaction(reaction)
    reaction.add_metabolites({me.metabolites.get_by_id(model_met_id): -1})
    # set bounds on exchanges
    if reaction.id.startswith("EX_") and met_id in source_amounts.index:
        reaction.lower_bound = -source_amounts.amount[met_id]

Set bounds on boundary reactions

## Build ribosome and RNA Polymerase

In [11]:
ribosome_complex = ComplexData("ribosome", me)
ribosome_components = ribosome_complex.stoichiometry
for i in ["b3851", "b3854", "b3855"]:
    ribosome_components["RNA_" + i] = 1
# 30S Listed as [rpsA -rpsU], sra 
# [rplA-rplF],  rplI, [rplK-rplY],
# [rpmA-rpmJ]

for i in ["b0911", "b0169", "b3314", "b3296", "b3303", "b4200", "b3341",
          "b3306", "b3230", "b3321", "b3297", "b3342", "b3298", "b3307", 
          "b3165", "b2609", "b3311", "b4202", "b3316", "b0023", "b3065",
          "b1480"]:
    ribosome_components["protein_" + i] = 1
    
# 50S listed as [rplA-rplF],rplJ, rplI, rplK [rplM-rplY], [rpmA-rpmJ]    
for i in ["b3984", "b3317", "b3320", "b3319", "b3308", "b3305", 
          "b3958", "b4203", "b3983", "b3231", "b3310", "b3301",
          "b3313", "b3294", "b3304", "b2606", "b1716", "b3186",
          "b3315", "b3318", "b3309", "b2185", 
          "b3185", "b3637", "b3312", "b3302", "b3936", "b1089", 
          "b3636", "b3703", "b1717", "b3299"]:
    ribosome_components["protein_" + i] = 1
# [rplJ, 2(2[rplL7])]
ribosome_components["protein_" + "b3986"] = 4

ribosome_complex.create_complex_formation()

Created <TranslatedGene protein_b1089 at 0x7ff19c2cd890> in <ComplexFormation formation_ribosome at 0x7ff19c2cd810>
Created <TranslatedGene protein_b3637 at 0x7ff19c2cd8d0> in <ComplexFormation formation_ribosome at 0x7ff19c2cd810>
Created <TranslatedGene protein_b3636 at 0x7ff19c2cd910> in <ComplexFormation formation_ribosome at 0x7ff19c2cd810>
Created <TranslatedGene protein_b3304 at 0x7ff19c2cd750> in <ComplexFormation formation_ribosome at 0x7ff19c2cd810>
Created <TranslatedGene protein_b4200 at 0x7ff19c2cd790> in <ComplexFormation formation_ribosome at 0x7ff19c2cd810>
Created <TranslatedGene protein_b1716 at 0x7ff19c2cd4d0> in <ComplexFormation formation_ribosome at 0x7ff19c2cd810>
Created <TranslatedGene protein_b1717 at 0x7ff19c2cd510> in <ComplexFormation formation_ribosome at 0x7ff19c2cd810>
Created <TranslatedGene protein_b0023 at 0x7ff19c2cd550> in <ComplexFormation formation_ribosome at 0x7ff19c2cd810>
Created <TranslatedGene protein_b3306 at 0x7ff19c2cd590> in <ComplexForm

In [12]:
RNAP_complex = ComplexData("RNA_Polymerase", me)
RNAP_components = RNAP_complex.stoichiometry
# Core RNA Polymerase Enzyme
for i in {"b3295" : "rpoA", "b3988" : "rpoC", "b3987" : "rpoB"}:
    if i == "b3295":
        RNAP_components["protein_" + i] = 2
    else:
        RNAP_components["protein_" + i] = 1

RNAP_complex.create_complex_formation()

Created <TranslatedGene protein_b3295 at 0x7ff19c2c3f90> in <ComplexFormation formation_RNA_Polymerase at 0x7ff19c2c3c10>
Created <TranslatedGene protein_b3988 at 0x7ff19c2c3fd0> in <ComplexFormation formation_RNA_Polymerase at 0x7ff19c2c3c10>
Created <TranslatedGene protein_b3987 at 0x7ff19c2c3f10> in <ComplexFormation formation_RNA_Polymerase at 0x7ff19c2c3c10>


### Add Transcription/Translation from the genbank file

In [13]:
build_reactions_from_genbank(me, join(ecoli_files_dir, "NC_000913.2.gb"))

TODO deal with selenocystine


Add a dummy protein in as well

In [14]:
dna_sequence = "ATG" + "TTT" * 5 + "TAT"*5+ "ACG"*5 + "GAT" *5 + "AGT"*5 + "TAA"
add_transcription_reaction(me, "dummy", {"dummy"}, dna_sequence)
me.add_metabolites(TranslatedGene("protein_" + "dummy"))
translation = TranslationReaction("translation_" + "dummy")
me.add_reaction(translation)
translation.translation_data = TranslationData("dummy", me, "RNA_dummy", "protein_dummy")
translation.translation_data.compute_sequence_from_DNA(dna_sequence)
translation.update()

complex_data = ComplexData("CPLX_dummy", me)
complex_data.stoichiometry = {}
complex_data.stoichiometry["protein_" + "dummy"] = 1
complex_data.create_complex_formation()

## Associate the tRNA synthetases

The tRNA charging reactions were automatically added when loading the genome from the genbank file. However, the charging reactions still need to be made aware of the tRNA synthetases which are responsible.

In [15]:
with open(join(ecoli_files_dir, "amino_acid_tRNA_synthetase.json"), "rb") as infile:
    aa_synthetase_dict = json.load(infile)
for data in me.tRNA_data:
    data.synthetase = str(aa_synthetase_dict[data.amino_acid])

### Add in complex Formation with modifications

In [16]:
# ME_complex_dict is a dict of {'complex_id': ['bnum(count)']}
ME_complex_dict = get_complex_to_bnum_dict()
# some entries in the file need to be renamed
renames = {"MnmE_": "b3706", "MnmG_": "b3741", "YheM_": "b3344", "YheL_": "b3343", "YheN_": "b3345"}
rna_components = {"b3123"}  # component id should have RNA_ instead of protein_

for cplx, value in ME_complex_dict.iteritems():
    complex_data = ComplexData(cplx, me)
    for gene in value:
        stoichiometry = gene[6]
        bnum = gene[0:5]
        comp_id = "RNA_" + bnum if bnum in rna_components \
            else "protein_" + renames.get(bnum, bnum)
        try:
            complex_data.stoichiometry[comp_id] = float(stoichiometry)
        except:
            complex_data.stoichiometry[comp_id] = float(1)

In [17]:
# {modified_complex_id: ['unmodified_complex_id', {component_id: stoic}]
modification_dict = get_protein_modification_dict(generic=True)
for mod_cplx_id, mod_complex_info in iteritems(modification_dict):
    unmod_cplx_id, mods = mod_complex_info
    unmod_cplx = me.complex_data.get_by_id(unmod_cplx_id)
    cplx = ComplexData(mod_cplx_id, me)
    cplx.stoichiometry = unmod_cplx.stoichiometry
    cplx.translocation = unmod_cplx.translocation
    cplx.chaperones = unmod_cplx.chaperones
    if len(set(mod_cplx_id.split("_mod_")[1:])) == len(mods):
        for mod_comp, mod_count in iteritems(mods):
            mod_id = "mod_" + mod_comp
            try:
                mod = me.modification_data.get_by_id(mod_id)
            except:
                mod = ModificationData(mod_id, me)
                mod.stoichiometry = {mod_comp: -1}
            cplx.modifications[mod_id] = -mod_count
    else:
        print "TODO:", mod_cplx_id

TODO: hRNAP_mod_1:generic_divalent_mod_2:generic_divalent
TODO: GltX_mono_mod_generic_divalent_mod_1:generic_divalent


Some modifications are enzyme-catalyzed

TODO: Fe-S enzyme-catalyzed modifications

In [18]:
# two different reactiond can add a lipoate modification.
# We create a separate ModificationData for each one
lipo = me.modification_data.get_by_id("mod_lipo_c")
alt_lipo = ModificationData("mod_lipo_c_alt", me)
alt_lipo.stoichiometry = lipo.stoichiometry

lipo.stoichiometry = {"lipoamp_c": -1, "amp_c": 1}
lipo.enzyme = 'EG11796-MONOMER'
lipo.keff = 65.

alt_lipo.stoichiometry = {'EG50003-MONOMER_mod_pan4p_mod_oc':-1,
                          'amet_c':-2,
                          'lipoamp_c':-1,
                          'met__L_c':2,
                          'dad__5_c':2,
                          'CPLX0-782_mod_2:4fe4s':-1,
                          'CPLX0-782_mod_1:2fe2s_mod_1:4fe4s':1,
                          'fe2_c':2,
                          'h_c':-2,
                          'nad_c':-1,
                          'nadh_c':1,
                          'EG50003-MONOMER_mod_pan4p':1}
alt_lipo.enzyme = 'EG11591-MONOMER'
alt_lipo.keff = 65.

for cplx_data in lipo.get_complex_data():
    alt_cplx_data = ComplexData(cplx_data.id + "alt", me)
    alt_cplx_data.complex_id = cplx_data.complex_id
    alt_cplx_data.translocation = cplx_data.translocation
    alt_cplx_data.chaperones = cplx_data.chaperones
    alt_cplx_data.modifications = cplx_data.modifications
    alt_cplx_data.modifications[alt_lipo.id] = \
        alt_cplx_data.modifications.pop(lipo.id)

In [19]:
# todo bmocogdp_c mods
mod_catalysts = {'CPLX0-1762':'G6712-MONOMER', # FE-S modification
                 'TMAOREDUCTI-CPLX':'EG12195-MONOMER',
                 'DIMESULFREDUCT-CPLX':'G6849-MONOMER',
                 'NITRATREDUCTA-CPLX':'NARJ-MONOMER', 
                 'NITRATREDUCTZ-CPLX':'NARW-MONOMER',
                 'NAP-CPLX':'NAPD-MONOMER',
                 'NAPAB-CPLX_NAPC-MONOMER':'NAPD-MONOMER'}

In [20]:
target_list = ['4fe4s_c', 'LI_c', '2fe2s_c', '3fe4s_c', 'pqq_c',
               'hemed_c', 'acetyl_c', 'NiFeCoCN2_c', 'SH_c',
               '23bpg_c', 'dpm_c', 'tl_c', 'sheme_c']
target_list.extend(
    ['EG50003-MONOMER_mod_pan4p', 'CPLX0-782_mod_2:4fe4s',
     'EG50003-MONOMER_mod_pan4p_mod_oc', 'CPLX0-782_mod_1:2fe2s_mod_1:4fe4s'])

for met_id in target_list:
    r = cobra.Reaction("EX_" + met_id)
    me.add_reaction(r)
    r.reaction = met_id + " <=> "

unknown metabolite '4fe4s_c' created
unknown metabolite 'LI_c' created
unknown metabolite '2fe2s_c' created
unknown metabolite '3fe4s_c' created
unknown metabolite 'pqq_c' created
unknown metabolite 'hemed_c' created
unknown metabolite 'acetyl_c' created
unknown metabolite 'NiFeCoCN2_c' created
unknown metabolite 'SH_c' created
unknown metabolite '23bpg_c' created
unknown metabolite 'dpm_c' created
unknown metabolite 'tl_c' created
unknown metabolite 'EG50003-MONOMER_mod_pan4p' created
unknown metabolite 'CPLX0-782_mod_2:4fe4s' created
unknown metabolite 'EG50003-MONOMER_mod_pan4p_mod_oc' created
unknown metabolite 'CPLX0-782_mod_1:2fe2s_mod_1:4fe4s' created


Build all complex formation reactions

TODO: This shouldn't be prining out stuff. Fix them.

In [21]:
for cplx_data in me.complex_data:
    formation = cplx_data.formation
    if formation:
        formation.update()
    else:
        cplx_data.create_complex_formation()

Crutch reactions for mets that are blocked. TODO remove

## Associate Complexes with Reactions

In [22]:
# associate reaction id with the old ME complex id (including modifications)
rxnToModCplxDict = get_reaction_to_modified_complex(generic=True)

Fixed _DASH:  D__LACtex
Fixed _DASH:  L__LACD2
Fixed _DASH:  L__LACD3
Fixed _DASH:  D__LACt2pp
Fixed _DASH:  L__LACtex
Fixed _DASH:  L__LACt2rpp


In [23]:
for reaction_data in me.stoichiometric_data:
    # Some reactions have no complex associated. Determine if they are
    # spontaneous or orphan reactions.
    try:
        complexes = rxnToModCplxDict[reaction_data.id]
    except KeyError:
        # These are orphans catalyzed by a dummy
        if reaction_data.id == "dummy_reaction" or \
                not reaction_info.is_spontaneous[reaction_data.id]:
            complexes = ["CPLX_dummy"]
        # These are truly spontaneous
        else:
            complexes = [None]
    for complex_id in complexes:
        complex_data = me.complex_data.get_by_id(complex_id) if complex_id else None
        if reaction_data.lower_bound < 0:
            r = MetabolicReaction(reaction_data.id + "_REV_" + str(complex_id))
            me.add_reaction(r)
            r.keff = 65
            r.stoichiometric_data = reaction_data
            if complex_data is not None:
                r.complex_data = complex_data
            r.reverse = True
            r.update(create_new=True)
        if reaction_data.upper_bound > 0:
            r = MetabolicReaction(reaction_data.id + "_FWD_" + str(complex_id))
            me.add_reaction(r)
            r.keff = 65
            r.stoichiometric_data = reaction_data
            if complex_data is not None:
                r.complex_data = complex_data
            r.reverse = False
            r.update(create_new=True)

Created <Metabolite CPLX0-7817_mod_Oxidized at 0x7ff199f5cf90> in <MetabolicReaction RNDR1b3_FWD_RIBONUCLEOSIDE-DIP-REDUCTII-CPLX at 0x7ff199f5cf50>
Created <Metabolite GLUTAREDOXIN-MONOMER_mod_Oxidized at 0x7ff199f66050> in <MetabolicReaction RNDR1b1_FWD_RIBONUCLEOSIDE-DIP-REDUCTII-CPLX at 0x7ff199f5cfd0>
Created <Metabolite GRXC-MONOMER_mod_Oxidized at 0x7ff199f660d0> in <MetabolicReaction RNDR1b4_FWD_RIBONUCLEOSIDE-DIP-REDUCTII-CPLX at 0x7ff199f66090>
Created <Metabolite GRXB-MONOMER_mod_Oxidized at 0x7ff199f66250> in <MetabolicReaction RNDR1b2_FWD_RIBONUCLEOSIDE-DIP-REDUCTII-CPLX at 0x7ff199f66210>
Created <Metabolite EG50003-MONOMER_mod_pan4p_mod_3ha at 0x7ff199f66850> in <MetabolicReaction 3OAR401_REV_3-OXOACYL-ACP-REDUCT-MONOMER at 0x7ff199f66810>
Created <Metabolite EG50003-MONOMER_mod_pan4p_mod_act at 0x7ff199f66890> in <MetabolicReaction 3OAR401_REV_3-OXOACYL-ACP-REDUCT-MONOMER at 0x7ff199f66810>
Created <Metabolite EG50003-MONOMER_mod_pan4p_mod_ddca at 0x7ff199f66a10> in <Me

In [24]:
# This reaction is weird
me.reactions.get_by_id('CITLY-CPLX_2tpr3dpcoa_FWD_G6340-MONOMER').reaction

'4.27350427350427e-6*mu G6340-MONOMER + 2tpr3dpcoa --> '

Rebuild transcription and translation to use tRNA (now that tRNA synthetase complexes are in the model

In [25]:
for r in me.reactions:
    if isinstance(r, tRNAChargingReaction):
        r.update()
for r in me.reactions:
    if isinstance(r, TranslationReaction):
        r.update()
    if isinstance(r, TranscriptionReaction):
        r.update()
    if isinstance(r, MetabolicReaction):
        r.update()

Remove unused protein and mRNA to make the model solve faster (TODO remove unused complexes too)

In [26]:
for c_d in me.complex_data:
    c = c_d.complex
    if len(c.reactions) == 1:
        list(c.reactions)[0].delete(remove_orphans=True)
for p in me.metabolites.query("protein"):
    if len(p._reaction) == 1:
        list(p._reaction)[0].delete(remove_orphans=True)
for m in me.metabolites.query("RNA"):
    if len(m._reaction) == 1:
        list(m._reaction)[0].delete(remove_orphans=True)

This gives the total number of genes included

In [27]:
len(me.reactions.query("transcription"))

1499

## Attempt to set keffs

In [28]:
divalent_list = divalent_list
monovalent_list = ['_mod_k','_mod_na1']
from pickle import load
with open("test_keffs.pickle", "rb") as infile:
    old_keffs = load(infile)
keffs = {}

for keff, value in old_keffs.items():
    for i in divalent_list: 
        keff = keff.replace(i,'generic_divalent')
    for i in monovalent_list: 
        keff = keff.replace(i, '_mod_generic_monovalent')
    keffs[keff] = value
    
for r in me.reactions:
    if isinstance(r, MetabolicReaction) and r.complex_data is None:
        print r.id, "has no associated complex"
        continue
    if isinstance(r, MetabolicReaction) and r.complex_data.id != "CPLX_dummy":
        met_rxn = r
        key = met_rxn.id.replace("-", "_DASH_").replace("__", "_DASH_").replace(":","_COLON_")
        #key = met_rxn.id
        key = "keff_" + key.replace("_FWD_", "_").replace("_REV_", "_")

        matches = [i for i in keffs if key in i]
        # get the direction
        if met_rxn.reverse:
            matches = [i for i in matches if i.endswith("_reverse_priming_keff")]
        else:
            matches = [i for i in matches if i.endswith("_forward_priming_keff")]
        if len(matches) == 1:
            met_rxn.keff = keffs[matches[0]]
        elif len(matches) > 0:
            if len(matches) == len([i for i in matches if key + "_mod_"]):
                met_rxn.keff = keffs[matches[0]]
            else:
                print key, len(matches)
        else:  # len(matches) == 0
            print "no keff found for", key

G5SADs_FWD_None has no associated complex
ACONIs_REV_None has no associated complex
ACONIs_FWD_None has no associated complex
CPGNexs_FWD_None has no associated complex
ARBTNexs_FWD_None has no associated complex
DATPHs_FWD_None has no associated complex
METOX1s_FWD_None has no associated complex
FE3HOXexs_FWD_None has no associated complex
no keff found for keff_METAT_S_DASH_ADENMETSYN_DASH_CPLX_mod_4_COLON_generic_monovalent_mod_1_COLON_generic_divalent
ATPHs_FWD_None has no associated complex
RFAMPtex_REV_None has no associated complex
RFAMPtex_FWD_None has no associated complex
O2tpp_REV_None has no associated complex
O2tpp_FWD_None has no associated complex
NOVBCNtex_REV_None has no associated complex
NOVBCNtex_FWD_None has no associated complex
MTHTHFSs_FWD_None has no associated complex
SO2tpp_REV_None has no associated complex
SO2tpp_FWD_None has no associated complex
DMSOtpp_REV_None has no associated complex
DMSOtpp_FWD_None has no associated complex
ETOHtrpp_REV_None has no 

## Solve

In [29]:
me.reactions.dummy_reaction_FWD_CPLX_dummy.objective_coefficient = 1.

In [30]:
expressions = compile_expressions(me)

In [31]:
solve_at_growth_rate(me, 0.1, compiled_expressions=expressions)

<Solution 0.10 at 0x7ff19b1ec090>

In [32]:
binary_search(me, min_mu=0, max_mu=1, mu_accuracy=1e-6,
              compiled_expressions=expressions)

0.0000000000000	1000.0
1.0000000000000	1000.0
2.0000000000000
1.5000000000000
1.2500000000000	1000.0
1.3750000000000	1000.0
1.4375000000000	1000.0
1.4687500000000	1000.0
1.4843750000000
1.4765625000000
1.4726562500000
1.4707031250000	378.387989813
1.4716796875000
1.4711914062500	107.541856465
1.4714355468750
1.4713134765625	39.8605834834
1.4713745117188	6.02448392326
1.4714050292969
1.4713897705078
1.4713821411133	1.79518411862
1.4713859558105
1.4713840484619	0.737866550447
1.4713850021362	0.209208873801
1.4713850021362	0.209208873801
completed in 1641.4 seconds and 24 iterations


<Solution 1.47 at 0x7ff19b8ca710>

In [33]:
me.compute_solution_error()

{'lower_bound_error': 1.2605343141381751e-32,
 'max_error': 1.2349149480783694e-13,
 'sum_error': 3.2202177589866029e-13,
 'upper_bound_error': 0}